# Gathering Historical Data

In the following notebook, we will gather historical data from 2000-2025 to be used for training the `PatchTST` time-series based transformer.
Multiple methods will be shown and explained such as, data preprocessing, data cleaning, and normalization.

## Import Data

We will import data from the FRED and yfinance that track multiple economic factors.

In [26]:
# Importing The Necessary Libraries
import pandas as pd
import numpy as np
np.NaN = np.nan # For pandas_ta
import yfinance as yf
import pandas_datareader as pdr
import pandas_ta as ta

In [27]:

# Define Tickers for stocks
SP_TICKER = '^GSPC'
DJ_TICKER = '^DJI'
NAS_TICKER = '^IXIC'
VIX_TICKER = '^VIX'

# Stock Futures 
SPF_TICKER = 'ES=F'
DJF_TICKER = 'YM=F'
NASF_TICKER = 'NQ=F'

# Define FRED data series codes
CPI_CODE = 'CPIAUCSL'
INTEREST_RATES_CODE = 'DFF'
UNEMPLOYMENT_RATES_CODE = 'UNRATE'
GDP_CODE = 'GDP'
M2_CODE = 'WM2NS'
INITIAL_CLAIMS_CODE = 'ICSA'
JOB_OPENINGS_CONSTRUCTION_CODE = 'JTS2300JOL'
JOB_OPENINGS_PRIVATE_CODE = 'JTS1000JOL'
JOB_OPENINGS_NF_CODE = 'JTSJOL'
PCE_CODE = 'PCE'
CONSUMER_DEBT_CODE = 'REVOLSL'
TDSP_CODE = 'TDSP'
CDSP_CODE = 'CDSP'
T10Y2Y_CODE = 'T10Y2Y'
T10YFF_CODE = 'T10YFF'
INDUSTRIAL_PROD_CODE = 'INDPRO'
CAPACITY_PROD_CODE = 'TCU'
RETAIL_SALES_CODE = 'RSAFS'
PERSONAL_SAVINGS_CODE = 'PSAVERT'
MORTGAGE_30_YEAR_CODE = 'MORTGAGE30US'
NEW_PRIV_HOUSING_CODE = 'HOUST'
PERSONAL_INCOME_CODE = 'PI'

# Define Domain of Data
START_DATE = '2002-04-05'
END_DATE = '2025-04-07'


In [28]:
# This function acts as a way to grab data from the fred based on the given 
# code and domain
def get_data_frame(fred_code, start_date, end_date):
    df = pdr.get_data_fred(fred_code, start=start_date, end=end_date)
    df = df.reset_index()
    df[fred_code] = df[fred_code].ffill().bfill()
    return df

In [29]:
# This function acts as a way to grab data for stocks based on the given 
# ticker and domain
def get_stock_data(ticker, start_date, end_date):
    copy_df = yf.download(ticker, start=start_date, end=end_date).ffill().bfill()
    copy_df = copy_df.reset_index()
    copy_df = copy_df[['Date', 'High', 'Low', 'Open', 'Close', 'Volume']]
    copy_df.columns = copy_df.columns.get_level_values(0)
    copy_df = copy_df.rename(columns={
           'Date'  : 'DATE',
           'High'  : f'{ticker} High', 
           'Low'   : f'{ticker} Low', 
           'Open'  : f'{ticker} Open', 
           'Close' : f'{ticker} Close', 
           'Volume': f'{ticker} Volume'
    })
    return copy_df

In [30]:
# S&P data
SP500 = get_stock_data(SP_TICKER, START_DATE, END_DATE)

# Dow Jones data
Dow_Jones = get_stock_data(DJ_TICKER, START_DATE, END_DATE)

# Nasdaq data
Nas = get_stock_data(NAS_TICKER, START_DATE, END_DATE)

# VIX data
Vix = get_stock_data(VIX_TICKER, START_DATE, END_DATE)

# S&P Futures
SPF = get_stock_data(SPF_TICKER, START_DATE, END_DATE)

# Dow Jones data
DWF = get_stock_data(DJF_TICKER, START_DATE, END_DATE)

# Nasdaq data
NASF = get_stock_data(NASF_TICKER, START_DATE, END_DATE)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [31]:
# Define DataFrames for FRED Data
cpi_df = get_data_frame(CPI_CODE, start_date=START_DATE, end_date=END_DATE)
interest_rates_df = get_data_frame(INTEREST_RATES_CODE, start_date=START_DATE, end_date=END_DATE)
unemployment_rates_df = get_data_frame(UNEMPLOYMENT_RATES_CODE, start_date=START_DATE, end_date=END_DATE)
gdp_df = get_data_frame(GDP_CODE, start_date=START_DATE, end_date=END_DATE)
m2_df = get_data_frame(M2_CODE, start_date=START_DATE, end_date=END_DATE)
initial_claims_df = get_data_frame(INITIAL_CLAIMS_CODE, start_date=START_DATE, end_date=END_DATE)
job_openings_construction_df = get_data_frame(JOB_OPENINGS_CONSTRUCTION_CODE, start_date=START_DATE, end_date=END_DATE)
job_openings_private_df = get_data_frame(JOB_OPENINGS_PRIVATE_CODE, start_date=START_DATE, end_date=END_DATE)
job_openings_nf_df = get_data_frame(JOB_OPENINGS_NF_CODE, start_date=START_DATE, end_date=END_DATE)
pce_df = get_data_frame(PCE_CODE, start_date=START_DATE, end_date=END_DATE)
consumer_debt_df = get_data_frame(CONSUMER_DEBT_CODE, start_date=START_DATE, end_date=END_DATE)
tdsp_df = get_data_frame(TDSP_CODE, start_date=START_DATE, end_date=END_DATE)
cdsp_df = get_data_frame(CDSP_CODE, start_date=START_DATE, end_date=END_DATE)
t10y2y_df = get_data_frame(T10Y2Y_CODE, start_date=START_DATE, end_date=END_DATE)
t10yff_df = get_data_frame(T10YFF_CODE, start_date=START_DATE, end_date=END_DATE)
industrial_prod_df = get_data_frame(INDUSTRIAL_PROD_CODE, start_date=START_DATE, end_date=END_DATE)
capacity_prod_df = get_data_frame(CAPACITY_PROD_CODE, start_date=START_DATE, end_date=END_DATE)
retail_sales_df = get_data_frame(RETAIL_SALES_CODE, start_date=START_DATE, end_date=END_DATE)
personal_savings_df = get_data_frame(PERSONAL_SAVINGS_CODE, start_date=START_DATE, end_date=END_DATE)
mortgage_30_year_df = get_data_frame(MORTGAGE_30_YEAR_CODE, start_date=START_DATE, end_date=END_DATE)
new_priv_housing_df = get_data_frame(NEW_PRIV_HOUSING_CODE, start_date=START_DATE, end_date=END_DATE)
personal_income_df = get_data_frame(PERSONAL_INCOME_CODE, start_date=START_DATE, end_date=END_DATE)

## Data Preprocessing

Here we will adjust values for inflation, and put all data frames into one

### Inflation Calculation

Inflation is defined in multiple ways, but we will use the following formula to calculate inflation using the average CPI per year

$$\text{Adjusted Amount} = \text{Past Year Dollar Amount} \times \frac{\text{Current Year CPI}}{\text{Past Year CPI}}$$

In [32]:
# Get the average CPI for each year to calculate inflation
CPI_yearly_amount = cpi_df.groupby(cpi_df['DATE'].dt.year)[CPI_CODE].mean().reset_index()

# For later calculations
CPI_2025 = CPI_yearly_amount[CPI_yearly_amount['DATE'] == 2025][CPI_CODE].values[0]

In [33]:
# Formula that returns a DataFrame that adjust contents for inflation
def adjust_for_inflation(df, features, CPI_yearly_amount):
    df = df.copy()

    # Get Year for both DF's
    df['Year'] = df['DATE'].dt.year
    CPI_yearly_amount = CPI_yearly_amount.rename(columns={'DATE' : 'Year'})

    # Add a new column to the df that has the CPI for each year
    df = pd.merge_asof(df, CPI_yearly_amount, on='Year', direction='forward')
                        
    # Adjust each value for inflation       
    for feature in features:
        df[feature] = df[feature] * (CPI_2025 / df[CPI_CODE])
        
    df = df.drop([CPI_CODE, 'Year'], axis=1)
        
    return df

In [34]:
SP500 = adjust_for_inflation(SP500, [f'{SP_TICKER} High', f'{SP_TICKER} Low', f'{SP_TICKER} Open', f'{SP_TICKER} Close'], CPI_yearly_amount)
Dow_Jones = adjust_for_inflation(Dow_Jones, [f'{DJ_TICKER} High', f'{DJ_TICKER} Low', f'{DJ_TICKER} Open', f'{DJ_TICKER} Close'], CPI_yearly_amount)
Nas_Inflation = adjust_for_inflation(Nas, [f'{NAS_TICKER} High', f'{NAS_TICKER} Low', f'{NAS_TICKER} Open', f'{NAS_TICKER} Close'], CPI_yearly_amount)
SPF = adjust_for_inflation(SPF, [f'{SPF_TICKER} High', f'{SPF_TICKER} Low', f'{SPF_TICKER} Open', f'{SPF_TICKER} Close'], CPI_yearly_amount)
DWF = adjust_for_inflation(DWF, [f'{DJF_TICKER} High', f'{DJF_TICKER} Low', f'{DJF_TICKER} Open', f'{DJF_TICKER} Close'], CPI_yearly_amount)
NASF = adjust_for_inflation(NASF, [f'{NASF_TICKER} High', f'{NASF_TICKER} Low', f'{NASF_TICKER} Open', f'{NASF_TICKER} Close'], CPI_yearly_amount)
pce_df = adjust_for_inflation(pce_df, [PCE_CODE], CPI_yearly_amount)
consumer_debt_df = adjust_for_inflation(consumer_debt_df, [CONSUMER_DEBT_CODE], CPI_yearly_amount)
retail_sales_df = adjust_for_inflation(retail_sales_df, [RETAIL_SALES_CODE], CPI_yearly_amount)
personal_income_df = adjust_for_inflation(personal_income_df, [PERSONAL_INCOME_CODE], CPI_yearly_amount)

### Combine all Data into One DataFrame

In this section we will combine all data based on the Date. For the different frequency of data we will fill missing gaps by filling forward with the previously received data

- For example:

    - GDP only updates quarterly while S&P 500 data is updates daily. The GDP will have missing values for each day, besides when a new report came out. 
    To adjust for this we will just fill the missing spots with what the previous GDP record was. 
    - Example below demonstraights this:

    | S&P Close | GDP |
    |----------|----------|
    | 5010 |   15.3 |
    | 5070 | NaN |
    | 5040 | NaN |

    **Gets Converted Too**

    | S&P Close | GDP |
    |----------|----------|
    | 5010 |   15.3 |
    | 5070 | 15.3 |
    | 5040 | 15.3 |


In [35]:
# This function merges the two dataFrames based on the date. Basically adds the col_name to the main_df
def add_data_based_on_date(main_df, new_data, col_name):
    return pd.merge_asof(
            main_df, 
            new_data,
            on='DATE',
            direction='forward' # How we fill the gaps when adding content
        )[col_name]

Adding Stock Markets

In [36]:
combined_df = pd.DataFrame()

# Add date for merging
combined_df['DATE'] = SP500['DATE']

# Adding Stock Markets
combined_df[f'{SP_TICKER} Close'] = add_data_based_on_date(combined_df, SP500, f'{SP_TICKER} Close')
combined_df[f'{SP_TICKER} High'] = add_data_based_on_date(combined_df, SP500, f'{SP_TICKER} High')
combined_df[f'{SP_TICKER} Low'] = add_data_based_on_date(combined_df, SP500, f'{SP_TICKER} Low')
combined_df[f'{SP_TICKER} Open'] = add_data_based_on_date(combined_df, SP500, f'{SP_TICKER} Open')
combined_df[f'{SP_TICKER} Volume'] = add_data_based_on_date(combined_df, SP500, f'{SP_TICKER} Volume')

combined_df[f'{DJ_TICKER} Close'] = add_data_based_on_date(combined_df, Dow_Jones, f'{DJ_TICKER} Close')
combined_df[f'{DJ_TICKER} High'] = add_data_based_on_date(combined_df, Dow_Jones, f'{DJ_TICKER} High')
combined_df[f'{DJ_TICKER} Low'] = add_data_based_on_date(combined_df, Dow_Jones, f'{DJ_TICKER} Low')
combined_df[f'{DJ_TICKER} Open'] = add_data_based_on_date(combined_df, Dow_Jones, f'{DJ_TICKER} Open')
combined_df[f'{DJ_TICKER} Volume'] = add_data_based_on_date(combined_df, Dow_Jones, f'{DJ_TICKER} Volume')

combined_df[f'{NAS_TICKER} Close'] = add_data_based_on_date(combined_df, Nas, f'{NAS_TICKER} Close')
combined_df[f'{NAS_TICKER} High'] = add_data_based_on_date(combined_df, Nas, f'{NAS_TICKER} High')
combined_df[f'{NAS_TICKER} Low'] = add_data_based_on_date(combined_df, Nas, f'{NAS_TICKER} Low')
combined_df[f'{NAS_TICKER} Open'] = add_data_based_on_date(combined_df, Nas, f'{NAS_TICKER} Open')
combined_df[f'{NAS_TICKER} Volume'] = add_data_based_on_date(combined_df, Nas, f'{NAS_TICKER} Volume')

combined_df[f'{VIX_TICKER} Close'] = add_data_based_on_date(combined_df, Vix, f'{VIX_TICKER} Close')
combined_df[f'{VIX_TICKER} High'] = add_data_based_on_date(combined_df, Vix, f'{VIX_TICKER} High')
combined_df[f'{VIX_TICKER} Low'] = add_data_based_on_date(combined_df, Vix, f'{VIX_TICKER} Low')
combined_df[f'{VIX_TICKER} Open'] = add_data_based_on_date(combined_df, Vix, f'{VIX_TICKER} Open')
combined_df[f'{VIX_TICKER} Volume'] = add_data_based_on_date(combined_df, Vix, f'{VIX_TICKER} Volume')

In [37]:
# Adding Stock Futures
# Adding Stock Markets
combined_df[f'{SPF_TICKER} Close'] = add_data_based_on_date(combined_df, SPF, f'{SPF_TICKER} Close')
combined_df[f'{SPF_TICKER} High'] = add_data_based_on_date(combined_df, SPF, f'{SPF_TICKER} High')
combined_df[f'{SPF_TICKER} Low'] = add_data_based_on_date(combined_df, SPF, f'{SPF_TICKER} Low')
combined_df[f'{SPF_TICKER} Open'] = add_data_based_on_date(combined_df, SPF, f'{SPF_TICKER} Open')
combined_df[f'{SPF_TICKER} Volume'] = add_data_based_on_date(combined_df, SPF, f'{SPF_TICKER} Volume')

combined_df[f'{DJF_TICKER} Close'] = add_data_based_on_date(combined_df, DWF, f'{DJF_TICKER} Close')
combined_df[f'{DJF_TICKER} High'] = add_data_based_on_date(combined_df, DWF, f'{DJF_TICKER} High')
combined_df[f'{DJF_TICKER} Low'] = add_data_based_on_date(combined_df, DWF, f'{DJF_TICKER} Low')
combined_df[f'{DJF_TICKER} Open'] = add_data_based_on_date(combined_df, DWF, f'{DJF_TICKER} Open')
combined_df[f'{DJF_TICKER} Volume'] = add_data_based_on_date(combined_df, DWF, f'{DJF_TICKER} Volume')

combined_df[f'{NASF_TICKER} Close'] = add_data_based_on_date(combined_df, NASF, f'{NASF_TICKER} Close')
combined_df[f'{NASF_TICKER} High'] = add_data_based_on_date(combined_df, NASF, f'{NASF_TICKER} High')
combined_df[f'{NASF_TICKER} Low'] = add_data_based_on_date(combined_df, NASF, f'{NASF_TICKER} Low')
combined_df[f'{NASF_TICKER} Open'] = add_data_based_on_date(combined_df, NASF, f'{NASF_TICKER} Open')
combined_df[f'{NASF_TICKER} Volume'] = add_data_based_on_date(combined_df, NASF, f'{NASF_TICKER} Volume')

Adding Monetary Policy

In [38]:
combined_df[f'{CPI_CODE}'] = add_data_based_on_date(combined_df, cpi_df, CPI_CODE)
combined_df[f'{INTEREST_RATES_CODE}'] = add_data_based_on_date(combined_df, interest_rates_df, INTEREST_RATES_CODE)
combined_df[f'{UNEMPLOYMENT_RATES_CODE}'] = add_data_based_on_date(combined_df, unemployment_rates_df, UNEMPLOYMENT_RATES_CODE)
combined_df[f'{GDP_CODE}'] = add_data_based_on_date(combined_df, gdp_df, GDP_CODE)
combined_df[f'{M2_CODE}'] = add_data_based_on_date(combined_df, m2_df, M2_CODE)
combined_df[f'{INITIAL_CLAIMS_CODE}'] = add_data_based_on_date(combined_df, initial_claims_df, INITIAL_CLAIMS_CODE)
combined_df[f'{JOB_OPENINGS_CONSTRUCTION_CODE}'] = add_data_based_on_date(combined_df, job_openings_construction_df, JOB_OPENINGS_CONSTRUCTION_CODE)
combined_df[f'{JOB_OPENINGS_PRIVATE_CODE}'] = add_data_based_on_date(combined_df, job_openings_private_df, JOB_OPENINGS_PRIVATE_CODE)
combined_df[f'{JOB_OPENINGS_NF_CODE}'] = add_data_based_on_date(combined_df, job_openings_nf_df, JOB_OPENINGS_NF_CODE)
combined_df[f'{PCE_CODE}'] = add_data_based_on_date(combined_df, pce_df, PCE_CODE)
combined_df[f'{CONSUMER_DEBT_CODE}'] = add_data_based_on_date(combined_df, consumer_debt_df, CONSUMER_DEBT_CODE)
combined_df[f'{TDSP_CODE}'] = add_data_based_on_date(combined_df, tdsp_df, TDSP_CODE)
combined_df[f'{CDSP_CODE}'] = add_data_based_on_date(combined_df, cdsp_df, CDSP_CODE)
combined_df[f'{T10Y2Y_CODE}'] = add_data_based_on_date(combined_df, t10y2y_df, T10Y2Y_CODE)
combined_df[f'{T10YFF_CODE}'] = add_data_based_on_date(combined_df, t10yff_df, T10YFF_CODE)
combined_df[f'{INDUSTRIAL_PROD_CODE}'] = add_data_based_on_date(combined_df, industrial_prod_df, INDUSTRIAL_PROD_CODE)
combined_df[f'{CAPACITY_PROD_CODE}'] = add_data_based_on_date(combined_df, capacity_prod_df, CAPACITY_PROD_CODE)
combined_df[f'{RETAIL_SALES_CODE}'] = add_data_based_on_date(combined_df, retail_sales_df, RETAIL_SALES_CODE)
combined_df[f'{PERSONAL_SAVINGS_CODE}'] = add_data_based_on_date(combined_df, personal_savings_df, PERSONAL_SAVINGS_CODE)
combined_df[f'{MORTGAGE_30_YEAR_CODE}'] = add_data_based_on_date(combined_df, mortgage_30_year_df, MORTGAGE_30_YEAR_CODE)
combined_df[f'{NEW_PRIV_HOUSING_CODE}'] = add_data_based_on_date(combined_df, new_priv_housing_df, NEW_PRIV_HOUSING_CODE)
combined_df[f'{PERSONAL_INCOME_CODE}'] = add_data_based_on_date(combined_df, personal_income_df, PERSONAL_INCOME_CODE)

In [39]:
# To account for any missing values
combined_df = combined_df.ffill()
combined_df.to_csv('../data/model_data/PreNorm_Full_Data.csv')

### Apply Window Based Normlization (Time-Series Based)

The way we will apply normilzation is as follows:

- Using scikit learns `RobustScaler` for better outlier handling for features by:
    - Calculating the scalar using the IQR, the range between the 75th and 25th percentiles
        - Using the standard divation is more prone to outliers
        - Using the minimum and maximum values could be outliers
        - **Uses *Interquartile range* that includes where most values fall**
    - Also uses the median rather than the mean
        - Using the mean is heavily influenced by outliers
        - X_min could be the outlier itself
        - **Uses the *median* which isn't effected by outliers as much**

- Will only scale based on **Past** data.
    - `window_size` will be our scale for how far we want to look back in time for normalization

In [22]:
from sklearn.preprocessing import RobustScaler
from tqdm import tqdm

def norm_data_st(df, col, window_size=90):
    df_scaled = df.copy()

    # Moving through each row starting at 
    # window_size and going to df length
    for i in tqdm(range(window_size, len(df))):

        # Grabs data from past
        time_window = df.iloc[i-window_size:i]

        # Grab current time
        current_time = df.iloc[i:i+1]

        # Create scaler on based on window
        robust_scaler = RobustScaler()
        robust_scaler.fit(time_window[col])

        # Apply scaler to current index feature
        scaled_values = robust_scaler.transform(current_time[col])
        df_scaled.loc[df.index[i], col] = scaled_values[0]
    return df_scaled

In [23]:
normalized_df = combined_df.copy()

# Our data will be normalized on a 2 year scale
WINDOW_SIZE = int(365 * 1.5) 

# Remove date from normalization
columns_to_normalize = combined_df.columns[1:len(combined_df)]

normalized_df = norm_data_st(normalized_df, columns_to_normalize, WINDOW_SIZE)

  0%|          | 0/5242 [00:00<?, ?it/s]/var/folders/dz/ls7s785d52j9jwwk1cpbk5fr0000gn/T/ipykernel_2801/3585957934.py:23: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '-0.7758399118780981' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_scaled.loc[df.index[i], col] = scaled_values[0]
/var/folders/dz/ls7s785d52j9jwwk1cpbk5fr0000gn/T/ipykernel_2801/3585957934.py:23: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '-0.8358186244674376' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_scaled.loc[df.index[i], col] = scaled_values[0]
/var/folders/dz/ls7s785d52j9jwwk1cpbk5fr0000gn/T/ipykernel_2801/3585957934.py:23: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '-0.53

In [24]:
# We need to drop the first 1.5 years as they weren't normalized
normalized_df = normalized_df[WINDOW_SIZE:len(normalized_df)]

In [25]:
normalized_df.to_csv('../data/model_data/PostNorm_Full_Data.csv')